# AVALIAÇÃO 3 - PROGRAMAÇÃO EM FINANÇAS
#### Professor: Julio Russo
#### Disciplina: Programação em Finanças
#### Turma: 2025.1
#### Alunos: Lucas dos Santos Marques, Luiz Fernando M. Meirinho , Victor A. S. de Souza e Victor Flávio P. Dornelos
#### Data: 25/06/2025

## 1 - Preparando os dados

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px
from tabulate import tabulate
import time

In [2]:
# Carteira brasileira
acoes_br = ['BBAS3.SA','VALE3.SA','EMBR3.SA','RANI3.SA']
# Baixar os dados
df_br = yf.download(acoes_br,auto_adjust=True,start='2014-01-01',end='2024-12-31')['Close']

# Salvando em CSV
df_br.to_csv('carteira_brasileira.csv')

[*********************100%***********************]  4 of 4 completed


In [3]:
# Carteira EUA corrigida (apenas tickers válidos)
acoes_eua = ['AAPL', 'JPM', 'AMZN', 'KO', 'BRK-B', 'XOM','TSLA']

# Baixar os dados de fechamento ajustado
df_eua = yf.download(acoes_eua, auto_adjust=True, start='2014-01-01', end='2024-12-31')['Close']

# Salvar em CSV
df_eua.to_csv('carteira_eua.csv')

[*********************100%***********************]  7 of 7 completed


## PARTE 1 - AÇÕES BRASIL

### QUESTÃO 1:
Formação de portfólio com ações brasileiras (mínimo 4 ativos). Não é necessário ter os mesmo pesos iniciais.


In [4]:
# Importando os dados
df_br = pd.read_csv('carteira_brasileira.csv', index_col='Date', parse_dates=True)
pesos_br = np.array([0.35, 0.35, 0.15, 0.15])  

### QUESTÃO 2:
Analisar retornos, volatilidade(desv.pad) dos ativos e da carteira

In [5]:
# Retornos logarítmicos diários
retornos_diarios = np.log(df_br / df_br.shift(1)).dropna()

# Indicadores individuais
retorno_anual = retornos_diarios.mean() * 252
retorno_acumulado = (df_br.iloc[-1] / df_br.iloc[0]) - 1
desvio_padrao_anual = retornos_diarios.std() * np.sqrt(252)
coef_variacao = desvio_padrao_anual / retorno_anual

In [6]:
# Tabela por ativo
tabela_ativos = pd.DataFrame({
    'Retorno Anual (%)': retorno_anual * 100,
    'Retorno Acumulado (%)': retorno_acumulado * 100,
    'Desvio Padrão Anual (%)': desvio_padrao_anual * 100,
    'Coef. de Variação': coef_variacao
})

In [7]:
# Retorno do portfólio
retorno_portfolio_diario = retornos_diarios @ pesos_br
retorno_anual_ptf = retorno_portfolio_diario.mean() * 252
retorno_acumulado_ptf = np.exp(retorno_portfolio_diario.cumsum())[-1] - 1
desvio_padrao_ptf = retorno_portfolio_diario.std() * np.sqrt(252)
coef_variacao_ptf = desvio_padrao_ptf / retorno_anual_ptf

/var/folders/wk/k6089y_d7hv_rwb2m8vbm9cr0000gn/T/ipykernel_23173/989489004.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  retorno_acumulado_ptf = np.exp(retorno_portfolio_diario.cumsum())[-1] - 1


In [8]:
# Adiciona à tabela
tabela_ativos.loc['Carteira'] = [
    retorno_anual_ptf * 100,
    retorno_acumulado_ptf * 100,
    desvio_padrao_ptf * 100,
    coef_variacao_ptf
]

In [9]:
# Arredondar para melhor visualização
tabela_ativos = tabela_ativos.round(2)

In [10]:
# Taxa livre de risco (ajuste conforme o país e período)
r_f = 0.01  # Exemplo: 12% a.a. no Brasil

# Sharpe dos ativos
sharpe_ativos = (retorno_anual - r_f) / desvio_padrao_anual

# Sharpe da carteira
sharpe_portfolio = (retorno_anual_ptf - r_f) / desvio_padrao_ptf

# Adiciona à tabela
tabela_ativos['Sharpe Ratio'] = sharpe_ativos
tabela_ativos.loc['Carteira', 'Sharpe Ratio'] = sharpe_portfolio

# Exibe com tabulate
from tabulate import tabulate
print("\n📊 Indicadores com Sharpe Ratio:\n")
print(tabulate(tabela_ativos.round(2), headers='keys', tablefmt='fancy_grid'))


📊 Indicadores com Sharpe Ratio:

╒══════════╤═════════════════════╤═════════════════════════╤═══════════════════════════╤═════════════════════╤════════════════╕
│          │   Retorno Anual (%) │   Retorno Acumulado (%) │   Desvio Padrão Anual (%) │   Coef. de Variação │   Sharpe Ratio │
╞══════════╪═════════════════════╪═════════════════════════╪═══════════════════════════╪═════════════════════╪════════════════╡
│ BBAS3.SA │               12.9  │                  305.52 │                     41.51 │                3.22 │           0.29 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ EMBR3.SA │               10.39 │                  208.57 │                     43.18 │                4.16 │           0.22 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ RANI3.SA │               10.97 │                  228.81 │          

### QUESTÃO 3:
Construção dos gráficos em plotly de retorno e gráfico PIE das participações relativas das ações no portfólio.

In [11]:
df_pie = df_br.copy()
df_br.columns = [col.replace('.SA', '') for col in df_br.columns]  # tira .SA
df_br = df_br.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Preço')

In [12]:
# Criar DataFrame
df_pie = pd.DataFrame({
    'Ativo': acoes_br,
    'Peso': pesos_br
})

# tirar. SA dos tickers
df_pie['Ativo'] = df_pie['Ativo'].str.replace('.SA', '')

In [13]:
# Cores personalizadas
cores_personalizadas_br = {
    'BBAS3': 'gold',         # amarelo
    'VALE3': 'red',          # vermelho
    'EMBR3': 'darkblue',     # azul escuro
    'RANI3': 'green',         # verde
    'Carteira': 'purple'     # roxo para a carteira
}


In [14]:
df_br = pd.read_csv('carteira_brasileira.csv', index_col='Date', parse_dates=True)

In [15]:
# Normalizar os ativos
df_norm = (df_br / df_br.iloc[0]) * 100
df_norm = df_norm.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Rentabilidade Normalizada (%)')


In [16]:
# Calcular o valor normalizado da carteira
pesos = np.array([0.35, 0.35, 0.15, 0.15])
port_br = df_br @ pesos
port_br_norm = (port_br / port_br.iloc[0]) * 100
df_port = port_br_norm.reset_index().rename(columns={0: 'Rentabilidade Normalizada (%)'})
df_port['Ativo'] = 'Carteira'

In [17]:
# Unir dados
df_total = pd.concat([df_norm, df_port], ignore_index=True)


In [18]:
df_plot =  df_br.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Preço')

In [19]:
import plotly.graph_objects as go

# --------- Tela Inicial ---------
tela_inicial = go.Indicator(
    title={"text": "Selecione um gráfico acima"},
    number={"font": {"color": "lightgray"}},
    domain={"x": [0, 1], "y": [0, 1]}
)

# --------- Pizza ---------
pizza = go.Pie(
    labels=[i for i in df_pie['Ativo']],
    values=df_pie['Peso'],
    name='Participações',
    marker_colors=[cores_personalizadas_br.get(i, 'gray') for i in df_pie['Ativo']],
    textinfo='percent+label',
    hole=0
)

# --------- Preço ---------
linhas_preco = [
    go.Scatter(
        x=df_plot[df_plot['Ativo'] == ativo]['Date'],
        y=df_plot[df_plot['Ativo'] == ativo]['Preço'],
        name=ativo.replace('.SA', ''),
        mode='lines',
        line=dict(color=cores_personalizadas_br.get(ativo.replace('.SA', ''), 'gray'))  # fallback: gray
    )
    for ativo in df_plot['Ativo'].unique()
]

# --------- Rentabilidade ---------
linhas_rent = [
    go.Scatter(
        x=df_total[df_total['Ativo'] == ativo]['Date'],
        y=df_total[df_total['Ativo'] == ativo]['Rentabilidade Normalizada (%)'],
        name=ativo.replace('.SA', ''),
        mode='lines',
        line=dict(color=cores_personalizadas_br.get(ativo.replace('.SA', ''), 'gray'))
    )
    for ativo in df_total['Ativo'].unique()
]

# --------- Layout ---------
layout = go.Layout(
    title='Gráficos Interativos de Ações',
    hovermode='x unified',
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    height=600,
    width=1500,
    updatemenus=[
        dict(
            type='buttons',
            direction='right',
            showactive=True,
            x=0,
            xanchor='left',
            y=1.05,
            yanchor='top',
            pad={'r': 10, 't': 10},
            buttons=[
                dict(label='Pizza',
                     method='update',
                     args=[{'visible': [False, True] + [False]*len(linhas_preco) + [False]*len(linhas_rent)},
                           {'title': 'Participações Relativas das Ações no Portfólio',
                            'xaxis': {'visible': False},
                            'yaxis': {'visible': False}}]),
                dict(label='Preço',
                     method='update',
                     args=[{'visible': [False, False] + [True]*len(linhas_preco) + [False]*len(linhas_rent)},
                           {'title': 'Evolução do Preço das Ações no Portfólio',
                            'xaxis': {'visible': True, 'rangeslider': {'visible': True}},
                            'yaxis': {'visible': True}}]),
                dict(label='Rentabilidade',
                     method='update',
                     args=[{'visible': [False, False] + [False]*len(linhas_preco) + [True]*len(linhas_rent)},
                           {'title': 'Rentabilidade Normalizada dos Ativos e da Carteira',
                            'xaxis': {'visible': True, 'rangeslider': {'visible': True}},
                            'yaxis': {'visible': True}}])
            ]
        )
    ]
)

# --------- Criar figura ---------
fig = go.Figure(data=[tela_inicial, pizza] + linhas_preco + linhas_rent, layout=layout)

# Visibilidade inicial: apenas a mensagem
visibility = [True, False] + [False]*(len(linhas_preco) + len(linhas_rent))
for i, trace in enumerate(fig.data):
    trace.visible = visibility[i]

fig.show()

## PARTE 2

### QUESTÃO 4
Formação de portfólio com ações americanas (mínimo 7 ativos). Não é necessário ter os mesmo pesos iniciais

In [20]:
# Importando os dados
df_eua = pd.read_csv('carteira_eua.csv', index_col='Date', parse_dates=True)
pesos_eua = [0.15, 0.15, 0.20, 0.10, 0.10, 0.15, 0.15]


### QUESTÃO 5
Analisar retornos, volatilidade(desv.pad) dos ativos e da carteira

In [21]:
# Retornos logarítmicos diários
retornos_diarios = np.log(df_eua / df_eua.shift(1)).dropna()

# Indicadores individuais
retorno_anual = retornos_diarios.mean() * 252
retorno_acumulado = (df_eua.iloc[-1] / df_eua.iloc[0]) - 1
desvio_padrao_anual = retornos_diarios.std() * np.sqrt(252)
coef_variacao = desvio_padrao_anual / retorno_anual

In [22]:
# Tabela por ativo
tabela_ativos = pd.DataFrame({
    'Retorno Anual (%)': retorno_anual * 100,
    'Retorno Acumulado (%)': retorno_acumulado * 100,
    'Desvio Padrão Anual (%)': desvio_padrao_anual * 100,
    'Coef. de Variação': coef_variacao
})

In [23]:
# Retorno do portfólio
retorno_portfolio_diario = retornos_diarios @ pesos_eua
retorno_anual_ptf = retorno_portfolio_diario.mean() * 252
retorno_acumulado_ptf = np.exp(retorno_portfolio_diario.cumsum())[-1] - 1
desvio_padrao_ptf = retorno_portfolio_diario.std() * np.sqrt(252)
coef_variacao_ptf = desvio_padrao_ptf / retorno_anual_ptf

/var/folders/wk/k6089y_d7hv_rwb2m8vbm9cr0000gn/T/ipykernel_23173/3228255680.py:4: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [24]:
# Adiciona à tabela
tabela_ativos.loc['Carteira'] = [
    retorno_anual_ptf * 100,
    retorno_acumulado_ptf * 100,
    desvio_padrao_ptf * 100,
    coef_variacao_ptf
]

In [25]:
# Arredondar para melhor visualização
tabela_ativos = tabela_ativos.round(2)

In [26]:
# Taxa livre de risco (ajuste conforme o país e período)
r_f = 0.01  # Exemplo: 12% a.a. no Brasil

# Sharpe dos ativos
sharpe_ativos = (retorno_anual - r_f) / desvio_padrao_anual

# Sharpe da carteira
sharpe_portfolio = (retorno_anual_ptf - r_f) / desvio_padrao_ptf

# Adiciona à tabela
tabela_ativos['Sharpe Ratio'] = sharpe_ativos
tabela_ativos.loc['Carteira', 'Sharpe Ratio'] = sharpe_portfolio

# Exibe com tabulate
from tabulate import tabulate
print("\n📊 Indicadores com Sharpe Ratio:\n")
print(tabulate(tabela_ativos.round(2), headers='keys', tablefmt='fancy_grid'))


📊 Indicadores com Sharpe Ratio:

╒══════════╤═════════════════════╤═════════════════════════╤═══════════════════════════╤═════════════════════╤════════════════╕
│          │   Retorno Anual (%) │   Retorno Acumulado (%) │   Desvio Padrão Anual (%) │   Coef. de Variação │   Sharpe Ratio │
╞══════════╪═════════════════════╪═════════════════════════╪═══════════════════════════╪═════════════════════╪════════════════╡
│ AAPL     │               24.45 │                 1363.36 │                     27.91 │                1.14 │           0.84 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ AMZN     │               21.95 │                 1012.14 │                     32.69 │                1.49 │           0.64 │
├──────────┼─────────────────────┼─────────────────────────┼───────────────────────────┼─────────────────────┼────────────────┤
│ BRK-B    │               12.28 │                  284.82 │          

### QUESTÃO 6
Construção dos gráficos em plotly de retorno e gráfico PIE das participações relativas das ações no portfólio.

In [27]:
df_pie = df_eua.copy()
df_eua = df_eua.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Preço')

In [28]:
# Criar DataFrame
df_pie = pd.DataFrame({
    'Ativo': acoes_eua,
    'Peso': pesos_eua
})

In [29]:
df_eua = pd.read_csv('carteira_eua.csv', index_col='Date', parse_dates=True)

In [30]:
# Normalizar os ativos
df_norm = (df_eua / df_eua.iloc[0]) * 100
df_norm = df_norm.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Rentabilidade Normalizada (%)')


In [31]:
# Calcular o valor normalizado da carteira
pesos = np.array([0.15, 0.15, 0.20, 0.10, 0.10, 0.15, 0.15])
port_eua = df_eua @ pesos
port_eua_norm = (port_eua / port_eua.iloc[0]) * 100
df_port = port_eua_norm.reset_index().rename(columns={0: 'Rentabilidade Normalizada (%)'})
df_port['Ativo'] = 'Carteira'

In [32]:
df_plot_eua =  df_eua.reset_index().melt(id_vars='Date', var_name='Ativo', value_name='Preço')

In [33]:
# Unir dados
df_total = pd.concat([df_norm, df_port], ignore_index=True)


In [34]:
import plotly.graph_objects as go

# --------- Cores Personalizadas EUA ---------
cores_personalizadas_eua = {
    'AAPL': '#A2AAAD',    # Apple - cinza
    'AMZN': '#FF9900',    # Amazon - laranja
    'BRK-B': '#0033A0',   # Berkshire - azul
    'JPM': '#0074A2',     # JPMorgan - azul claro
    'KO': '#E41F26',      # Coca-Cola - vermelho
    'TSLA': '#CC0000',    # Tesla - vermelho escuro
    'XOM': '#0054A4'      # Exxon - azul
}

# --------- Tela Inicial: Indicador com texto ---------
tela_inicial = go.Indicator(
    title={"text": "<b>Selecione um gráfico acima</b>"},
    number={"font": {"color": "lightgray"}},
    domain={"x": [0, 1], "y": [0, 1]}
)

# --------- Gráfico de Pizza ---------
pizza = go.Pie(
    labels=df_pie['Ativo'],
    values=df_pie['Peso'],
    name='Participações',
    marker_colors=[cores_personalizadas_eua.get(i, 'gray') for i in df_pie['Ativo']],
    textinfo='percent+label',
    hole=0
)

# --------- Gráfico de Preço ---------
linhas_preco = [
    go.Scatter(
        x=df_plot_eua[df_plot_eua['Ativo'] == ativo]['Date'],
        y=df_plot_eua[df_plot_eua['Ativo'] == ativo]['Preço'],
        name=ativo,
        mode='lines',
        line=dict(color=cores_personalizadas_eua.get(ativo, 'gray'))
    )
    for ativo in df_plot_eua['Ativo'].unique()
]

# --------- Gráfico de Rentabilidade ---------
linhas_rent = [
    go.Scatter(
        x=df_total[df_total['Ativo'] == ativo]['Date'],
        y=df_total[df_total['Ativo'] == ativo]['Rentabilidade Normalizada (%)'],
        name=ativo,
        mode='lines',
        line=dict(color=cores_personalizadas_eua.get(ativo, 'gray'))
    )
    for ativo in df_total['Ativo'].unique()
]

# --------- Layout com Botões ---------
layout = go.Layout(
    title='Gráficos Interativos de Ações',
    hovermode='x unified',
    xaxis=dict(visible=False),
    yaxis=dict(visible=False),
    height=600,
    width=1500,
    updatemenus=[
        dict(
            type='buttons',
            direction='right',
            showactive=True,
            x=0,
            xanchor='left',
            y=1.05,
            yanchor='top',
            pad={'r': 10, 't': 10},
            buttons=[
                dict(label='Pizza',
                     method='update',
                     args=[{'visible': [False, True] + [False]*len(linhas_preco) + [False]*len(linhas_rent)},
                           {'title': 'Participações Relativas das Ações no Portfólio',
                            'xaxis': {'visible': False},
                            'yaxis': {'visible': False}}]),
                dict(label='Preço',
                     method='update',
                     args=[{'visible': [False, False] + [True]*len(linhas_preco) + [False]*len(linhas_rent)},
                           {'title': 'Evolução do Preço das Ações no Portfólio',
                            'xaxis': {'visible': True, 'rangeslider': {'visible': True}},
                            'yaxis': {'visible': True}}]),
                dict(label='Rentabilidade',
                     method='update',
                     args=[{'visible': [False, False] + [False]*len(linhas_preco) + [True]*len(linhas_rent)},
                           {'title': 'Rentabilidade Normalizada dos Ativos e da Carteira',
                            'xaxis': {'visible': True, 'rangeslider': {'visible': True}},
                            'yaxis': {'visible': True}}])
            ]
        )
    ]
)

# --------- Criar Figura ---------
fig = go.Figure(data=[tela_inicial, pizza] + linhas_preco + linhas_rent, layout=layout)

# Visibilidade inicial: somente a mensagem
visibility = [True, False] + [False]*(len(linhas_preco) + len(linhas_rent))
for i, trace in enumerate(fig.data):
    trace.visible = visibility[i]

# Exibir
fig.show()